In [4]:
from agential.cog.agent.critic import CriticAgent
from langchain_community.tools.google_serper.tool import GoogleSerperRun
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    # Prompts.
    CRITIC_POT_INSTRUCTION_GSM8K,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K, 
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    # Examples.
    GSM8K_FEWSHOT_EXAMPLES_POT,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
)


import warnings
warnings.filterwarnings("ignore")

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [ ]:
from agential.cog.functional.critic import _prompt_agent

question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = _prompt_agent(
    llm=llm, question=question, examples=GSM8K_FEWSHOT_EXAMPLES_POT, prompt=CRITIC_POT_INSTRUCTION_GSM8K
)

In [5]:
from agential.cog.functional.critic import _prompt_critique
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = """clips_sold_in_april = 48
clips_sold_in_may = clips_sold_in_april / 30
answer = clips_sold_in_april + clips_sold_in_may"""

critique = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique="",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("Here's")[0]

<CRITIC PROMPT==========================================================================>
Question: Janet hires six employees. Four of them are warehouse workers who make $15/hour, and the other two are managers who make $20/hour. Janet has to pay 10% of her workers' salaries in FICA taxes. If everyone works 25 days a month and 8 hours a day, how much does Janet owe total for their wages and taxes for one month?
```python
num_of_warehouse_workers = 4
num_of_managers = 2
wage_of_warehouse_workers = 15
wage_of_managers = 20
num_of_days = 25
num_of_hours = 8
total_hours = num_of_days * num_of_hours
total_wage = num_of_warehouse_workers * wage_of_warehouse_workers * total_hours + num_of_managers * wage_of_managers * total_hours
answer = total_wage * 1.1
```
Execution: Done
Output: answer = 22000.0

What's the problem with the above code?

1. The total wages and taxes should be positive, 22000 > 0, it's reasonable.

2. Let's check the code:

> # Janet hires six employees. Four of them are w

In [ ]:
responses = [
    code,
    "The code provided is incorrect.\n\nThe issue lies in the calculation of the number of clips sold in May. It is stated that Natalia sold half as many clips in May compared to April, which means she sold 48/2 = 24 clips in May. The code incorrectly calculates the number of clips sold in May by dividing the number of clips sold in April by 30.\n\nHere's the corrected version of the code:\n\n```python\nclips_sold_in_april = 48\nclips_sold_in_may = clips_sold_in_april / 2  # Natalia sold half as many clips in May\nanswer = clips_sold_in_april + clips_sold_in_may\n```"
]

In [6]:
better = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique=critique + "\n\n" + "Here's a better solution:\n```python\n",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("```")[0]

<CRITIC PROMPT==========================================================================>
Question: Janet hires six employees. Four of them are warehouse workers who make $15/hour, and the other two are managers who make $20/hour. Janet has to pay 10% of her workers' salaries in FICA taxes. If everyone works 25 days a month and 8 hours a day, how much does Janet owe total for their wages and taxes for one month?
```python
num_of_warehouse_workers = 4
num_of_managers = 2
wage_of_warehouse_workers = 15
wage_of_managers = 20
num_of_days = 25
num_of_hours = 8
total_hours = num_of_days * num_of_hours
total_wage = num_of_warehouse_workers * wage_of_warehouse_workers * total_hours + num_of_managers * wage_of_managers * total_hours
answer = total_wage * 1.1
```
Execution: Done
Output: answer = 22000.0

What's the problem with the above code?

1. The total wages and taxes should be positive, 22000 > 0, it's reasonable.

2. Let's check the code:

> # Janet hires six employees. Four of them are w

In [8]:
print(better)

clips_sold_in_april = 48
clips_sold_in_may = clips_sold_in_april / 2
total_clips_sold = clips_sold_in_april + clips_sold_in_may
answer = total_clips_sold


In [ ]:
"""
         | with tool | without tool |
PoT      |           |      o       |
"""

In [ ]:
agent = CriticAgent(llm=llm, mode="code_interpreter", search=search)

In [ ]:
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"

import warnings
warnings.filterwarnings("ignore")

out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    use_interpreter_tool=True
)

In [ ]:
out